In [61]:
import os
import cv2
import csv
import numpy as np
from skimage import measure
import joblib
from tensorflow.keras.models import load_model



MODEL_PATH = ''
IMAGE_PATH = ''
target_size = (512, 512)

def segment_image(original_image, mask_image_path):


    original_image = original_image[0]

    #mask_image = cv2.resize(mask_image, (512, 512))
    mask_image = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)
    mask_image = cv2.resize(mask_image, (512, 512))


    # Make sure the mask is binary
    print(original_image.shape)
    print(mask_image.shape)
    _, binary_mask = cv2.threshold(mask_image, 0, 255, cv2.THRESH_BINARY)

    segmented_image = cv2.bitwise_and(original_image, original_image, mask=binary_mask)
    return segmented_image
    # cv2.imwrite(output_image_path, segmented_image)
    # return True


def predict_segmentation(model, preprocessed_image):

    prediction = model.predict(preprocessed_image)
    prediction = np.squeeze(prediction, axis=0)
    prediction = (prediction > 0.5).astype(np.uint8)
    return prediction


def calculate_average_rgb(image):
    '''
    This function calculates the average RGB values of an image.

    Args:
        image_path (str): The path to the image.

    Returns:
        numpy.ndarray: The average RGB values as a NumPy array.
    '''

    average_rgb = np.mean(image, axis=(0, 1))
    return average_rgb


def blur_image(image):
    '''
    This function blurs an image using a Gaussian blur and saves the blurred image.

    Args:

    Returns:
        bool: True if successful, False otherwise.
    '''
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    _, binary = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)

    labels = measure.label(binary, connectivity=2)

    properties = measure.regionprops(labels)
    largest_region = max(properties, key=lambda x: x.area)

    return largest_region

def preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image


def read_and_reshape_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = np.expand_dims(image, axis=0)
    print(image.shape)
    return image




In [62]:

def image_to_mean_rgb(file_names, body_parts, age, gender):
    '''['tongue', 'right_fingernail', 'left_fingernail', 'left_palm', 'right_palm', 'left_eye', 'right_eye']'''
    # average_rgb_values = []
    results = {}
    for i in range(7):
        filename = file_names[i]
        body_part = body_parts[i]
        model_name = ''
        print(body_part)

        # Load the appropriate pre-trained model
        if body_part.endswith("eye"):
            model_name = 'eyelid_model.h5'
        elif body_part.endswith("fingernail"):
            model_name = 'fingernail_model.h5'
        elif body_part.endswith("palm"):
            model_name = 'palm_model.h5'
        else:
            continue

        # Load image and model
        image_path = os.path.join(IMAGE_PATH, filename)
        model_path = os.path.join(MODEL_PATH, model_name)
        print(model_name)

        model = load_model(model_path)
        preprocessed_image = preprocess_image(image_path, target_size)
        prediction = predict_segmentation(model, preprocessed_image)


        # If the body part is not a fingernail, blur the image using gaussian blur
        if not body_part.endswith("fingernail"):
            image = prediction * 255
            blurred = cv2.GaussianBlur(image, (5, 5), 0)
            _, binary = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)
            labels = measure.label(binary, connectivity=2)
            properties = measure.regionprops(labels)
            if len(properties) == 0:
                print("No regions found")
                largest_region_mask = np.zeros_like(binary)
            else:
                largest_region = max(properties, key=lambda x: x.area)
                largest_region_mask = np.zeros_like(binary)
                largest_region_mask[labels == largest_region.label] = 255
            mask = largest_region_mask
        else:
            mask = prediction * 255

        cv2.imwrite(os.path.join('mask', filename), mask)
        print('blur done')


        # Segment the image
        original_image = read_and_reshape_image(image_path, target_size)
        print(original_image.shape)
        segmented_image = segment_image(original_image, os.path.join('mask', filename))
        cv2.imwrite(os.path.join('cropped', filename), segmented_image)
        print('segmentation done')


        # calculate average rgb
        average_rgb = calculate_average_rgb(segmented_image)
        # average_rgb_values.append(average_rgb)
        results[body_part] = average_rgb
        results['age'] = age
        results['gender'] = gender
    return results


#if "__name__" == "__main__":
values = image_to_mean_rgb(['tongue.jpg', 'right_nail.jpg', 'left_nail.jpg', 'left_palm.jpg', 'right_palm.jpg', 'left_eye.jpg', 'right_eye.jpg'],
                    ['tongue', 'right_fingernail', 'left_fingernail', 'left_palm', 'right_palm', 'left_eye', 'right_eye'], 22, 'male')
print(values)
print("Done")

tongue
right_fingernail
fingernail_model.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step


blur done
(1, 512, 512, 3)
(1, 512, 512, 3)
(512, 512, 3)
(512, 512)
segmentation done
left_fingernail
fingernail_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


blur done
(1, 512, 512, 3)
(1, 512, 512, 3)
(512, 512, 3)
(512, 512)
segmentation done
left_palm
palm_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
blur done
(1, 512, 512, 3)
(1, 512, 512, 3)
(512, 512, 3)
(512, 512)
segmentation done
right_palm
palm_model.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


blur done
(1, 512, 512, 3)
(1, 512, 512, 3)
(512, 512, 3)
(512, 512)
segmentation done
left_eye
eyelid_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


blur done
(1, 512, 512, 3)
(1, 512, 512, 3)
(512, 512, 3)
(512, 512)
segmentation done
right_eye
eyelid_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
blur done
(1, 512, 512, 3)
(1, 512, 512, 3)
(512, 512, 3)
(512, 512)
segmentation done
{'right_fingernail': array([2.2915535 , 2.94833374, 3.73516846]), 'age': 22, 'gender': 'male', 'left_fingernail': array([2.51485443, 4.4309845 , 5.31111526]), 'left_palm': array([18.03853607, 28.34124756, 33.01542282]), 'right_palm': array([ 9.72921371, 20.49105835, 29.70488739]), 'left_eye': array([0.00291824, 0.01586533, 0.05227661]), 'right_eye': array([0.30593872, 0.44629288, 0.82648087])}
Done


In [66]:
sum(values['right_fingernail'])

8.975055694580078

In [67]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization

In [68]:
xgb_model = xgb.XGBRegressor()
xgb_model.load_model("xgb_model.json")
print("Model loaded successfully!")

Model loaded successfully!


In [70]:
def preprocess_input(input_dict):
    combined_left_eye = sum(input_dict['left_eye'])/ 3
    combined_left_palm = sum(input_dict['right_palm']) / 3
    combined_right_nail = sum(input_dict['right_fingernail']) / 3

    features = np.array([
        combined_left_eye,
        combined_left_palm,
        combined_right_nail,
        values['right_fingernail'][0],
        values['right_fingernail'][1],
        values['right_fingernail'][2]
    ]).reshape(1, -1)
    return features

processed_input = preprocess_input(values)

predicted_hb_level = xgb_model.predict(processed_input)
print(f"Predicted Hemoglobin Level: {predicted_hb_level[0]:.2f} mg/dl")

Predicted Hemoglobin Level: 12.74 mg/dl
